In [1]:
import os
import pandas as pd
import itertools
import csv
from os import listdir
from os.path import isfile, join
from tba_invest_etl.utils import sql_manager

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
def get_table_schema(table_name, sql):

    table_name = "prices_wrds"

    q = """                              
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns
    WHERE table_name = %s;
    """

    cur = sql.cursor
    cur.execute(q, (table_name,))
    table_schema_list = cur.fetchall()
    table_schema = pd.DataFrame(table_schema_list, columns=["column_name", "data_type", "is_nullable"])

    return table_schema
    
def convert_integer(df, table_schema):
    col_int = table_schema.loc[table_schema.data_type == 'integer']['column_name']
    for col in col_int:
        df.loc[~df[col].isnull(), col] = df.loc[~df[col].isnull(), col].astype(float)

def convert_numeric(df, table_schema):
    col_numeric = table_schema.loc[table_schema.data_type.isin(['numeric', 'real'])]['column_name']
    for col in col_numeric:
        data = df.loc[~df[col].isnull(), col]
        try:
            data = data.str.replace(',', '')
        except:
            pass
        df.loc[~df[col].isnull(), col] = data.astype(float)

def convert_date_columns(df):
    col_dt = [col for col in df.columns if col.endswith('dt')]
    for col in col_dt:
        df.loc[~df[col].isnull(), col] = df.loc[~df[col].isnull(), col].astype(str).str[:8]

In [3]:
path = "data/crsp_prices.csv"
#path = "data/fin_ratios.csv"

In [4]:
def row_count(path):
    with open(path) as f:
        for i, l in enumerate(f):
            pass
    return i

In [5]:
#n_rows = row_count(path)
n_rows = 101150729
n_rows

101150729

In [146]:
with open(path) as f:
    cols = next(itertools.islice(csv.reader(f), 0, 1))
cols

['PERMNO',
 'PrimaryExch',
 'SecurityNm',
 'Ticker',
 'PERMCO',
 'IssuerNm',
 'YYYYMMDD',
 'DlyCalDt',
 'DlyDelFlg',
 'DlyPrc',
 'DlyPrcFlg',
 'DlyCap',
 'DlyCapFlg',
 'DlyPrevPrc',
 'DlyPrevPrcFlg',
 'DlyPrevDt',
 'DlyPrevCap',
 'DlyPrevCapFlg',
 'DlyRet',
 'DlyRetx',
 'DlyRetI',
 'DlyRetMissFlg',
 'DlyRetDurFlg',
 'DlyOrdDivAmt',
 'DlyNonOrdDivAmt',
 'DlyFacPrc',
 'DlyDistRetFlg',
 'DlyVol',
 'DlyClose',
 'DlyLow',
 'DlyHigh',
 'DlyBid',
 'DlyAsk',
 'DlyOpen',
 'DlyNumTrd',
 'DlyMMCnt',
 'DlyPrcVol',
 'ShrStartDt',
 'ShrEndDt',
 'ShrOut',
 'ShrSource',
 'ShrFacType',
 'ShrAdrFlg',
 'DisExDt',
 'DisSeqNbr',
 'DisOrdinaryFlg',
 'DisType',
 'DisFreqType',
 'DisPaymentType',
 'DisDetailType',
 'DisTaxType',
 'DisOrigCurType',
 'DisDivAmt',
 'DisFacPr',
 'DisFacShr',
 'DisDeclareDt',
 'DisRecordDt',
 'DisPayDt',
 'DisPERMNO',
 'DisPERMCO']

In [147]:
sql = sql_manager.ManagerSQL()
table_name = "prices_wrds"
table_schema = get_table_schema(table_name, sql)
table_schema.head()

,column_name,data_type,is_nullable
0,permno,character varying,YES
1,primaryexch,character varying,YES
2,securitynm,text,YES
3,ticker,character varying,YES
4,permco,character varying,YES


In [148]:
table_schema.data_type.value_counts()

character varying    35
numeric              14
real                  9
text                  2
Name: data_type, dtype: int64

In [6]:
local_folder = "data/crsp_prices"
filename = "crsp_prices_{}.csv"

In [150]:
# Divide csv into smaller csv
for i, chunk in enumerate(pd.read_csv(path, chunksize=1000000)):
    print(i)
    ii = f'00{i}'[-3:]
    print(ii)
    local_path = os.path.join(local_folder, filename.format(ii))
    chunk.columns = [col.lower() for col in chunk.columns]
    convert_integer(chunk, table_schema)
    convert_numeric(chunk, table_schema)
    convert_date_columns(chunk)
    chunk.to_csv(local_path, index=False)

/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (21,41,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


0
000


/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (21,30,41) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


1
001


/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (41,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


...

80
080
81
081
82
082


/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (9,13,21,24,28,29,30,31,32,33,41,45,46,47,48,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


83
083


/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (9,13,28,29,30,31,32,33,41,47) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


84
084
85
085
86
086
87
087
88
088
89
089
90
090
91
091
92
092
93
093
94
094
95
095
96
096
97
097
98
098
99
099
100
100
101
101


In [110]:
filenames = [f for f in listdir(local_folder) if isfile(join(local_folder, f))]
filenames_nbr = [int(filename[12:].replace('.csv','')) for filename in filenames]
filenames = [x for _, x in sorted(zip(filenames_nbr, filenames), key=lambda pair: pair[0])]
filenames[:5]

['crsp_prices_0.csv',
 'crsp_prices_1.csv',
 'crsp_prices_2.csv',
 'crsp_prices_3.csv',
 'crsp_prices_4.csv']

In [118]:
for filename in filenames[1:]:
    print(filename)
    local_path = os.path.join(local_folder, filename)
    df = pd.read_csv(local_path)
    print('before ', df.head())
    df.columns = [col.lower() for col in df.columns]
    convert_integer(df, table_schema)
    convert_numeric(df, table_schema)
    convert_date_columns(df)
    print('after ', df.head())
    sql.upload_df_chunks("prices_wrds", df, chunk_size=100)
    print(f'Done with {filename}')

crsp_prices_1.csv


/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (21,30,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


before     PERMNO PrimaryExch                         SecurityNm Ticker  PERMCO  \
0   10259           Q  SIGMA DESIGNS INC; COM NONE; CONS   SIGM    8187   
1   10259           Q  SIGMA DESIGNS INC; COM NONE; CONS   SIGM    8187   
2   10259           Q  SIGMA DESIGNS INC; COM NONE; CONS   SIGM    8187   
3   10259           Q  SIGMA DESIGNS INC; COM NONE; CONS   SIGM    8187   
4   10259           Q  SIGMA DESIGNS INC; COM NONE; CONS   SIGM    8187   

            IssuerNm  YYYYMMDD  DlyCalDt DlyDelFlg  DlyPrc DlyPrcFlg  \
0  SIGMA DESIGNS INC  20170731  20170731         N    6.50        TR   
1  SIGMA DESIGNS INC  20170801  20170801         N    6.55        TR   
2  SIGMA DESIGNS INC  20170802  20170802         N    6.50        TR   
3  SIGMA DESIGNS INC  20170803  20170803         N    6.25        TR   
4  SIGMA DESIGNS INC  20170804  20170804         N    6.25        TR   

       DlyCap DlyCapFlg  DlyPrevPrc DlyPrevPrcFlg   DlyPrevDt  DlyPrevCap  \
0  250,217.50        BP        

KeyboardInterrupt: 

In [115]:
col_int = table_schema.loc[table_schema.data_type == 'integer']['column_name']

In [116]:
df[col_int]
# numeric(14, 2): dlycap
# 

,permno,permco,dlymmcnt,shrout,dispermno,dispermco
0,10259,8187,50.0,38495.0,NaN,NaN
1,10259,8187,50.0,38495.0,NaN,NaN
2,10259,8187,50.0,38495.0,NaN,NaN
3,10259,8187,50.0,38495.0,NaN,NaN
4,10259,8187,50.0,38495.0,NaN,NaN
...,...,...,...,...,...,...
999995,10563,8472,28.0,11996.0,NaN,NaN
999996,10563,8472,28.0,11996.0,NaN,NaN
999997,10563,8472,26.0,11996.0,NaN,NaN
999998,10563,8472,28.0,11996.0,NaN,NaN


In [9]:
df.cusip.value_counts()

86835810    624
25365110    624
14912310    624
77571110    624
82654610    624
           ... 
37229820      1
14170630      1
87073T10      1
37193310      1
13910110      1
Name: cusip, Length: 30856, dtype: int64

In [11]:
local_path = os.path.join(local_folder, 'crsp_prices_000.csv')

In [12]:
local_path

'data/crsp_prices/crsp_prices_000.csv'

In [13]:
df = pd.read_csv(local_path)

/Users/nace/local/conda/envs/folio/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (21,41,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# permno int NOT NULL,
# date timestamp without time zone NOT NULL,
# ticker varchar(20),
# prc float NOT NULL,
# vol int,
# ret float NOT NULL,
# shrout int NOT NULL,

In [15]:
df.columns
# 'permno', 'primaryexch', 'ticker', 'issuernm', 'dlycaldt', 'dlyprc', 'dlyclose'

Index(['permno', 'primaryexch', 'securitynm', 'ticker', 'permco', 'issuernm',
       'yyyymmdd', 'dlycaldt', 'dlydelflg', 'dlyprc', 'dlyprcflg', 'dlycap',
       'dlycapflg', 'dlyprevprc', 'dlyprevprcflg', 'dlyprevdt', 'dlyprevcap',
       'dlyprevcapflg', 'dlyret', 'dlyretx', 'dlyreti', 'dlyretmissflg',
       'dlyretdurflg', 'dlyorddivamt', 'dlynonorddivamt', 'dlyfacprc',
       'dlydistretflg', 'dlyvol', 'dlyclose', 'dlylow', 'dlyhigh', 'dlybid',
       'dlyask', 'dlyopen', 'dlynumtrd', 'dlymmcnt', 'dlyprcvol', 'shrstartdt',
       'shrenddt', 'shrout', 'shrsource', 'shrfactype', 'shradrflg', 'disexdt',
       'disseqnbr', 'disordinaryflg', 'distype', 'disfreqtype',
       'dispaymenttype', 'disdetailtype', 'distaxtype', 'disorigcurtype',
       'disdivamt', 'disfacpr', 'disfacshr', 'disdeclaredt', 'disrecorddt',
       'dispaydt', 'dispermno', 'dispermco'],
      dtype='object')

In [31]:
cols_keep = ['permno', 'primaryexch', 'ticker', 'issuernm', 'dlycaldt', 'dlyprc', 
    'dlybid', 'dlyask', 'dlyvol', 'dlyret', 'shrout']

In [32]:
df[cols_keep]

,permno,primaryexch,ticker,issuernm,dlycaldt,dlyprc,dlybid,dlyask,dlyvol,dlyret,shrout
0,10000,Q,OMFGA,OPTIMUM MANUFACTURING INC,19860107,2.5625,2.375,2.750,1000.0,NaN,3680.0
1,10000,Q,OMFGA,OPTIMUM MANUFACTURING INC,19860108,2.5000,2.375,2.625,12800.0,-0.024390,3680.0
2,10000,Q,OMFGA,OPTIMUM MANUFACTURING INC,19860109,2.5000,2.375,2.625,1400.0,0.000000,3680.0
3,10000,Q,OMFGA,OPTIMUM MANUFACTURING INC,19860110,2.5000,2.375,2.625,8500.0,0.000000,3680.0
4,10000,Q,OMFGA,OPTIMUM MANUFACTURING INC,19860113,2.6250,2.500,2.750,5450.0,0.050000,3680.0
...,...,...,...,...,...,...,...,...,...,...,...
999995,10259,Q,SIGM,SIGMA DESIGNS INC,20170724,6.1500,6.100,6.150,128092.0,0.000000,38212.0
999996,10259,Q,SIGM,SIGMA DESIGNS INC,20170725,6.1000,6.100,6.150,203401.0,-0.008130,38212.0
999997,10259,Q,SIGM,SIGMA DESIGNS INC,20170726,6.7000,6.650,6.700,601663.0,0.098361,38212.0
999998,10259,Q,SIGM,SIGMA DESIGNS INC,20170727,6.5500,6.550,6.600,300517.0,-0.022388,38212.0


In [35]:
cols_drop = [col for col in df.columns if col not in cols_keep]

In [36]:
cols_drop

['securitynm',
 'permco',
 'yyyymmdd',
 'dlydelflg',
 'dlyprcflg',
 'dlycap',
 'dlycapflg',
 'dlyprevprc',
 'dlyprevprcflg',
 'dlyprevdt',
 'dlyprevcap',
 'dlyprevcapflg',
 'dlyretx',
 'dlyreti',
 'dlyretmissflg',
 'dlyretdurflg',
 'dlyorddivamt',
 'dlynonorddivamt',
 'dlyfacprc',
 'dlydistretflg',
 'dlyclose',
 'dlylow',
 'dlyhigh',
 'dlyopen',
 'dlynumtrd',
 'dlymmcnt',
 'dlyprcvol',
 'shrstartdt',
 'shrenddt',
 'shrsource',
 'shrfactype',
 'shradrflg',
 'disexdt',
 'disseqnbr',
 'disordinaryflg',
 'distype',
 'disfreqtype',
 'dispaymenttype',
 'disdetailtype',
 'distaxtype',
 'disorigcurtype',
 'disdivamt',
 'disfacpr',
 'disfacshr',
 'disdeclaredt',
 'disrecorddt',
 'dispaydt',
 'dispermno',
 'dispermco']

In [39]:
print("ALTER TABLE prices_wrds\n" + ",\n".join([f"DROP COLUMN {col}" for col in cols_drop[2:]]))

ALTER TABLE prices_wrds
DROP COLUMN yyyymmdd,
DROP COLUMN dlydelflg,
DROP COLUMN dlyprcflg,
DROP COLUMN dlycap,
DROP COLUMN dlycapflg,
DROP COLUMN dlyprevprc,
DROP COLUMN dlyprevprcflg,
DROP COLUMN dlyprevdt,
DROP COLUMN dlyprevcap,
DROP COLUMN dlyprevcapflg,
DROP COLUMN dlyretx,
DROP COLUMN dlyreti,
DROP COLUMN dlyretmissflg,
DROP COLUMN dlyretdurflg,
DROP COLUMN dlyorddivamt,
DROP COLUMN dlynonorddivamt,
DROP COLUMN dlyfacprc,
DROP COLUMN dlydistretflg,
DROP COLUMN dlyclose,
DROP COLUMN dlylow,
DROP COLUMN dlyhigh,
DROP COLUMN dlyopen,
DROP COLUMN dlynumtrd,
DROP COLUMN dlymmcnt,
DROP COLUMN dlyprcvol,
DROP COLUMN shrstartdt,
DROP COLUMN shrenddt,
DROP COLUMN shrsource,
DROP COLUMN shrfactype,
DROP COLUMN shradrflg,
DROP COLUMN disexdt,
DROP COLUMN disseqnbr,
DROP COLUMN disordinaryflg,
DROP COLUMN distype,
DROP COLUMN disfreqtype,
DROP COLUMN dispaymenttype,
DROP COLUMN disdetailtype,
DROP COLUMN distaxtype,
DROP COLUMN disorigcurtype,
DROP COLUMN disdivamt,
DROP COLUMN disfacpr,
DR

In [ ]:
ALTER TABLE prices_wrds  
DROP COLUMN securitynm,
DROP COLUMN permco;

In [18]:
df.loc[df.yyyymmdd != df.dlycaldt]

,permno,primaryexch,securitynm,ticker,permco,issuernm,yyyymmdd,dlycaldt,dlydelflg,dlyprc,dlyprcflg,dlycap,dlycapflg,dlyprevprc,dlyprevprcflg,dlyprevdt,dlyprevcap,dlyprevcapflg,dlyret,dlyretx,dlyreti,dlyretmissflg,dlyretdurflg,dlyorddivamt,dlynonorddivamt,dlyfacprc,dlydistretflg,dlyvol,dlyclose,dlylow,dlyhigh,dlybid,dlyask,dlyopen,dlynumtrd,dlymmcnt,dlyprcvol,shrstartdt,shrenddt,shrout,shrsource,shrfactype,shradrflg,disexdt,disseqnbr,disordinaryflg,distype,disfreqtype,dispaymenttype,disdetailtype,distaxtype,disorigcurtype,disdivamt,disfacpr,disfacshr,disdeclaredt,disrecorddt,dispaydt,dispermno,dispermco
